In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_excel("data\Telco_customer_churn_adapted_v2.xlsx")
df.sample(5)

Customer ID  Tenure Months Location Device Class        Games Product  \
5544         5544             42  Jakarta      Mid End                  Yes   
4418         4418              1  Jakarta      Mid End                   No   
363           363              1  Jakarta     High End                   No   
6045         6045              1  Jakarta      Low End  No internet service   
2575         2575             54  Bandung     High End                  Yes   

            Music Product    Education Product Call Center  \
5544                  Yes                  Yes          No   
4418                   No                   No          No   
363                    No                   No          No   
6045  No internet service  No internet service          No   
2575                  Yes                   No          No   

            Video Product            Use MyApp  Payment Method  \
5544                   No                   No           Debit   
4418                   No                   No  Digital Wallet   
363                   Yes                   No           Pulsa   
6045  No internet service  No internet service           Pulsa   
2575                  Yes                  Yes           Debit   

      Monthly Purchase (Thou. IDR) Churn Label   Longitude  Latitude  \
5544                        77.545          No  106.816666 -6.200000   
4418                        58.435          No  106.816666 -6.200000   
363                        102.960         Yes  106.816666 -6.200000   
6045                        25.675          No  106.816666 -6.200000   
2575                       135.330          No  107.609810 -6.914744   

      CLTV (Predicted Thou. IDR)  
5544                      3962.4  
4418                      4232.8  
363                       5671.9  
6045                      3988.4  
2575                      7533.5

# Statistik Umum

### Valuasi

In [8]:
# Distribusi Monthly Purchase
fig = px.histogram(df, x="CLTV (Predicted Thou. IDR)", nbins=20)
fig.show()

### Jumlah Customer

In [105]:
def pie(df, names, hover_variable):
    data = df[[names, "Customer ID"]].groupby(names).count().reset_index()
    hover_data = df[[names, hover_variable]].groupby(names).mean().reset_index()
    data[hover_variable] = round(hover_data[hover_variable], 3)

    fig = px.pie(data, values="Customer ID", names=names,
                title=f'Jumlah Customer Menurut {names}',
                hover_data=[hover_variable], labels={'Customer ID':'Jumlah Customer'})
    fig.update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

fig = pie(df, names = "Location", hover_variable = "Monthly Purchase (Thou. IDR)")
fig.show()

### Tenure

In [183]:
def tenure_months_distribution(y, agg):
    mapper = {
        0: "Less than 1 year",
        1: "1-2 years",
        2: "2-3 years",
        3: "3-4 years",
        4: "4-5 years",
        5: "5-6 years",
        6: "6-7 years"
    }

    if y == "Count":
        temp = pd.DataFrame((df["Tenure Months"] // 12).replace(mapper).value_counts()).reset_index()
        temp.columns = ["Tenure Months", "Count"]
    elif y != "Count":
        temp = df[["Tenure Months", y]]
        temp["Tenure Months"] = (temp["Tenure Months"] // 12).replace(mapper)

        if agg == "Sum":
            temp = temp.groupby("Tenure Months").sum().reset_index()
        elif agg == "Mean":
            temp = temp.groupby("Tenure Months").mean().reset_index()
        else:
            temp = temp.groupby("Tenure Months").median().reset_index()
            
    fig = px.bar(temp, y="Tenure Months", x = y,
                category_orders={"Tenure Months": list(mapper.values())},
                title = "Tenure Months Distribution")
    return fig

tenure_months_distribution(y="Count", agg="Median")

In [ ]:
def payment_method(y, agg="Mean", hue=None):
    if y == "Count":
        temp = df["Payment Method"].value_counts().reset_index()
        temp.columns = ["Payment Method", "Count"]
    elif y != "Count":
        if "Monthly" in y:
            y = "Monthly Purchase (Thou. IDR)"
        else:
            y = "CLTV (Predicted Thou. IDR)"
        temp = df[["Payment Method", y]].groupby("Payment Method").sum().reset_index()

        if agg == "Sum":
            temp = temp.groupby("Payment Method").sum().reset_index()
        elif agg == "Mean":
            temp = temp.groupby("Payment Method").mean().reset_index()
        else:
            temp = temp.groupby("Payment Method").median().reset_index()

    if hue:
        fig = px.bar(temp, y=y, x = "Payment Method", color=hue,
                    title = "Payment Method Distribution")
    else:
        fig = px.bar(temp, y=y, x = "Payment Method",
                    title = "Payment Method Distribution")
    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0), height=400)
    return fig
    
temp = df[["Payment Method", "Location", "Monthly Purchase (Thou. IDR)"]]
temp = temp.groupby(["Payment Method", "Location"]).mean().reset_index()

fig = px.bar(temp, y=y, x = "Payment Method", color="Location",
             title = "Payment Method Distribution", barmode="group")
fig.show()

payment_method(y="Count", agg="Sum", hue="Location")

In [55]:
temp = df[["Payment Method", "Location", "Monthly Purchase (Thou. IDR)"]]
temp = temp.groupby(["Payment Method", "Location"]).mean().reset_index()

fig = px.bar(temp, y=y, x = "Payment Method", color="Location",
             title = "Payment Method Distribution", barmode="group")
fig.show()

In [66]:
def payment_method(df, y="Count", agg="Mean", hue=None):
    if y == "Count":
        temp = df["Payment Method"].value_counts().reset_index()
        temp.columns = ["Payment Method", "Count"]
        if hue:
            temp = df[["Payment Method", hue]].groupby(["Payment Method", hue]).size().reset_index()
            temp.columns = ["Payment Method", hue, "Count"]
    else:
        if "Monthly" in y:
            y = "Monthly Purchase (Thou. IDR)"
        else:
            y = "CLTV (Predicted Thou. IDR)"
        if hue:
            temp = df[["Payment Method", hue, y]].groupby(["Payment Method", hue]).agg(agg.lower()).reset_index()
        else:
            temp = df[["Payment Method", y]].groupby("Payment Method").agg(agg.lower()).reset_index()

    if hue:
        fig = px.bar(temp, x="Payment Method", y=y, color=hue, title="Payment Method Distribution", barmode="group")
    else:
        fig = px.bar(temp, x="Payment Method", y=y, title="Payment Method Distribution")

    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0), height=400)
    return fig

payment_method(df, y="Monthly Purchase (Predicted Thou. IDR)", agg="Sum")
